# Lab 19: Advanced Cloud Security AI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab19_cloud_security.ipynb)

Build AI-powered multi-cloud security tools for detecting sophisticated cloud-native attacks.

## Learning Objectives
- AWS CloudTrail log analysis for attack detection
- Azure Activity Log and Sentinel integration
- GCP Cloud Audit Log analysis
- Multi-cloud event normalization
- Privilege escalation chain detection
- IAM persistence and backdoor detection
- Resource hijacking (cryptomining, data exfiltration)
- Serverless attack patterns
- LLM-powered threat analysis and response

## Cloud Attack Patterns Covered

This lab detects sophisticated cloud attacks:
1. **IAM Persistence** - Backdoor users, access keys, role chaining
2. **Privilege Escalation** - Policy manipulation, role assumption
3. **Defense Evasion** - CloudTrail disabling, log deletion
4. **Credential Access** - Secrets Manager, Parameter Store theft
5. **Discovery** - Resource enumeration, metadata service abuse
6. **Exfiltration** - S3 data theft, snapshot sharing
7. **Impact** - Cryptomining, resource destruction

In [ ]:
!pip install pandas anthropic -q

In [ ]:
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict

## 1. AWS CloudTrail Analysis

In [ ]:
# Comprehensive CloudTrail events simulating a sophisticated cloud attack

# Attack simulation: Compromised IAM credentials leading to data exfiltration
CLOUDTRAIL_ATTACK_SEQUENCE = [
    # Initial Access - Compromised credentials (from phishing or leaked keys)
    {
        "eventTime": "2024-01-15T08:00:00Z",
        "eventName": "ConsoleLogin",
        "eventSource": "signin.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "developer-john"},
        "sourceIPAddress": "203.0.113.50",  # Unusual IP
        "userAgent": "Mozilla/5.0 (Windows NT 10.0)",
        "responseElements": {"ConsoleLogin": "Success"},
        "additionalEventData": {"MFAUsed": "No", "LoginTo": "https://console.aws.amazon.com"}
    },
    
    # Discovery - Account enumeration
    {
        "eventTime": "2024-01-15T08:05:00Z",
        "eventName": "ListUsers",
        "eventSource": "iam.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "developer-john"},
        "sourceIPAddress": "203.0.113.50",
        "requestParameters": {}
    },
    {
        "eventTime": "2024-01-15T08:05:30Z",
        "eventName": "ListRoles",
        "eventSource": "iam.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "developer-john"},
        "sourceIPAddress": "203.0.113.50",
        "requestParameters": {}
    },
    {
        "eventTime": "2024-01-15T08:06:00Z",
        "eventName": "ListBuckets",
        "eventSource": "s3.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "developer-john"},
        "sourceIPAddress": "203.0.113.50",
    },
    {
        "eventTime": "2024-01-15T08:06:30Z",
        "eventName": "DescribeInstances",
        "eventSource": "ec2.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "developer-john"},
        "sourceIPAddress": "203.0.113.50",
    },
    
    # Persistence - Create backdoor user and access keys
    {
        "eventTime": "2024-01-15T08:10:00Z",
        "eventName": "CreateUser",
        "eventSource": "iam.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "developer-john"},
        "sourceIPAddress": "203.0.113.50",
        "requestParameters": {"userName": "svc-backup-2024"},
        "responseElements": {"user": {"userName": "svc-backup-2024", "userId": "AIDA..."}}
    },
    {
        "eventTime": "2024-01-15T08:10:30Z",
        "eventName": "CreateAccessKey",
        "eventSource": "iam.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "developer-john"},
        "sourceIPAddress": "203.0.113.50",
        "requestParameters": {"userName": "svc-backup-2024"},
        "responseElements": {"accessKey": {"accessKeyId": "AKIA...", "status": "Active"}}
    },
    
    # Privilege Escalation - Attach admin policy
    {
        "eventTime": "2024-01-15T08:11:00Z",
        "eventName": "AttachUserPolicy",
        "eventSource": "iam.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "developer-john"},
        "sourceIPAddress": "203.0.113.50",
        "requestParameters": {
            "userName": "svc-backup-2024",
            "policyArn": "arn:aws:iam::aws:policy/AdministratorAccess"
        }
    },
    
    # Defense Evasion - Disable CloudTrail (attempt)
    {
        "eventTime": "2024-01-15T08:15:00Z",
        "eventName": "StopLogging",
        "eventSource": "cloudtrail.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "svc-backup-2024"},
        "sourceIPAddress": "198.51.100.25",  # Different IP - using new credentials
        "requestParameters": {"name": "main-trail"},
        "errorCode": "AccessDenied",
        "errorMessage": "User: arn:aws:iam::123456789:user/svc-backup-2024 is not authorized"
    },
    {
        "eventTime": "2024-01-15T08:15:30Z",
        "eventName": "DeleteTrail",
        "eventSource": "cloudtrail.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "svc-backup-2024"},
        "sourceIPAddress": "198.51.100.25",
        "requestParameters": {"name": "secondary-trail"},
        "errorCode": "AccessDenied"
    },
    
    # Credential Access - Secrets Manager
    {
        "eventTime": "2024-01-15T08:20:00Z",
        "eventName": "ListSecrets",
        "eventSource": "secretsmanager.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "svc-backup-2024"},
        "sourceIPAddress": "198.51.100.25",
    },
    {
        "eventTime": "2024-01-15T08:20:30Z",
        "eventName": "GetSecretValue",
        "eventSource": "secretsmanager.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "svc-backup-2024"},
        "sourceIPAddress": "198.51.100.25",
        "requestParameters": {"secretId": "prod/database/credentials"}
    },
    {
        "eventTime": "2024-01-15T08:21:00Z",
        "eventName": "GetSecretValue",
        "eventSource": "secretsmanager.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "svc-backup-2024"},
        "sourceIPAddress": "198.51.100.25",
        "requestParameters": {"secretId": "prod/api/keys"}
    },
    
    # Data Exfiltration - S3 bucket access
    {
        "eventTime": "2024-01-15T08:25:00Z",
        "eventName": "GetBucketAcl",
        "eventSource": "s3.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "svc-backup-2024"},
        "sourceIPAddress": "198.51.100.25",
        "requestParameters": {"bucketName": "company-confidential-data"}
    },
    {
        "eventTime": "2024-01-15T08:25:30Z",
        "eventName": "GetObject",
        "eventSource": "s3.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "svc-backup-2024"},
        "sourceIPAddress": "198.51.100.25",
        "requestParameters": {"bucketName": "company-confidential-data", "key": "customer-database.sql"}
    },
    {
        "eventTime": "2024-01-15T08:26:00Z",
        "eventName": "GetObject",
        "eventSource": "s3.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "svc-backup-2024"},
        "sourceIPAddress": "198.51.100.25",
        "requestParameters": {"bucketName": "company-confidential-data", "key": "financial-reports-2024.xlsx"}
    },
    
    # Exfiltration - Share snapshot externally
    {
        "eventTime": "2024-01-15T08:30:00Z",
        "eventName": "ModifySnapshotAttribute",
        "eventSource": "ec2.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "svc-backup-2024"},
        "sourceIPAddress": "198.51.100.25",
        "requestParameters": {
            "snapshotId": "snap-0123456789abcdef0",
            "createVolumePermission": {"add": {"items": [{"userId": "999999999999"}]}}
        }
    },
    
    # Impact - Cryptomining (spin up instances)
    {
        "eventTime": "2024-01-15T08:35:00Z",
        "eventName": "RunInstances",
        "eventSource": "ec2.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "svc-backup-2024"},
        "sourceIPAddress": "198.51.100.25",
        "requestParameters": {
            "instanceType": "p3.16xlarge",  # GPU instance for mining
            "minCount": 10,
            "maxCount": 10,
            "imageId": "ami-0123456789"
        }
    },
]

# Normal baseline activity (noise)
CLOUDTRAIL_NORMAL_ACTIVITY = [
    {
        "eventTime": "2024-01-15T07:00:00Z",
        "eventName": "ConsoleLogin",
        "eventSource": "signin.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "admin-sarah"},
        "sourceIPAddress": "192.168.1.100",  # Corporate IP
        "responseElements": {"ConsoleLogin": "Success"},
        "additionalEventData": {"MFAUsed": "Yes"}
    },
    {
        "eventTime": "2024-01-15T07:30:00Z",
        "eventName": "DescribeInstances",
        "eventSource": "ec2.amazonaws.com",
        "userIdentity": {"type": "IAMUser", "userName": "admin-sarah"},
        "sourceIPAddress": "192.168.1.100",
    },
    {
        "eventTime": "2024-01-15T09:00:00Z",
        "eventName": "PutObject",
        "eventSource": "s3.amazonaws.com",
        "userIdentity": {"type": "AssumedRole", "arn": "arn:aws:sts::123456789:assumed-role/deploy-role/i-0123456"},
        "sourceIPAddress": "10.0.1.50",  # Internal IP
        "requestParameters": {"bucketName": "deploy-artifacts", "key": "app-v2.1.0.zip"}
    },
    {
        "eventTime": "2024-01-15T10:00:00Z",
        "eventName": "Invoke",
        "eventSource": "lambda.amazonaws.com",
        "userIdentity": {"type": "Service", "invokedBy": "events.amazonaws.com"},
        "requestParameters": {"functionName": "scheduled-backup"}
    },
]

# Cloud attack detection rules
CLOUD_DETECTION_RULES = [
    {
        "name": "IAM User Creation with Admin Policy",
        "description": "New IAM user created and immediately granted admin privileges",
        "events": ["CreateUser", "AttachUserPolicy"],
        "policy_patterns": ["AdministratorAccess", "PowerUserAccess", "IAMFullAccess"],
        "severity": "CRITICAL",
        "mitre": "T1136.003",
        "window_minutes": 30
    },
    {
        "name": "CloudTrail Tampering Attempt",
        "description": "Attempt to disable or delete CloudTrail logging",
        "events": ["StopLogging", "DeleteTrail", "UpdateTrail"],
        "severity": "CRITICAL",
        "mitre": "T1562.008"
    },
    {
        "name": "Secrets Mass Retrieval",
        "description": "Multiple secrets accessed in short time window",
        "events": ["GetSecretValue", "GetParametersByPath"],
        "threshold": 5,
        "window_minutes": 10,
        "severity": "HIGH",
        "mitre": "T1552.001"
    },
    {
        "name": "S3 Bucket Data Exfiltration",
        "description": "Large number of object downloads from sensitive buckets",
        "events": ["GetObject"],
        "bucket_patterns": ["confidential", "sensitive", "private", "backup"],
        "threshold": 10,
        "window_minutes": 60,
        "severity": "HIGH",
        "mitre": "T1530"
    },
    {
        "name": "EBS Snapshot Sharing",
        "description": "Snapshot shared with external AWS account",
        "events": ["ModifySnapshotAttribute"],
        "severity": "HIGH",
        "mitre": "T1537"
    },
    {
        "name": "Cryptomining Instance Launch",
        "description": "GPU instances launched (potential cryptomining)",
        "events": ["RunInstances"],
        "instance_types": ["p2.", "p3.", "p4.", "g4.", "g5."],
        "severity": "HIGH",
        "mitre": "T1496"
    },
    {
        "name": "Console Login Without MFA",
        "description": "Console login without multi-factor authentication",
        "events": ["ConsoleLogin"],
        "mfa_required": True,
        "severity": "MEDIUM",
        "mitre": "T1078.004"
    },
    {
        "name": "Unusual Source IP",
        "description": "API calls from unexpected geographic location or IP",
        "baseline_ips": ["192.168.0.0/16", "10.0.0.0/8"],
        "severity": "MEDIUM",
        "mitre": "T1078.004"
    },
]

# Combine events
all_events = CLOUDTRAIL_ATTACK_SEQUENCE + CLOUDTRAIL_NORMAL_ACTIVITY
df = pd.DataFrame(all_events)

print(f"Loaded {len(all_events)} CloudTrail events")
print(f"  Attack events: {len(CLOUDTRAIL_ATTACK_SEQUENCE)}")
print(f"  Normal events: {len(CLOUDTRAIL_NORMAL_ACTIVITY)}")
print(f"\nUnique event names: {df['eventName'].nunique()}")
print(f"Unique users: {df['userIdentity'].apply(lambda x: x.get('userName', 'N/A')).nunique()}")
print(f"\nDetection rules loaded: {len(CLOUD_DETECTION_RULES)}")

## 2. Cloud Threat Detection Rules

In [ ]:
# Cloud security detection rules
DETECTION_RULES = [
    {
        "name": "IAM Persistence",
        "events": ["CreateAccessKey", "CreateUser", "AttachUserPolicy"],
        "severity": "HIGH",
        "mitre": "T1136.003"
    },
    {
        "name": "Privilege Escalation",
        "events": ["AttachUserPolicy", "AttachRolePolicy", "PutUserPolicy"],
        "policy_patterns": ["AdministratorAccess", "PowerUserAccess"],
        "severity": "CRITICAL",
        "mitre": "T1078.004"
    },
    {
        "name": "Resource Enumeration",
        "events": ["DescribeInstances", "ListBuckets", "DescribeSecurityGroups"],
        "threshold": 10,  # More than 10 in 5 minutes
        "severity": "MEDIUM",
        "mitre": "T1580"
    },
]

def detect_cloud_threats(events: List[Dict], rules: List[Dict]) -> List[Dict]:
    """Apply detection rules to CloudTrail events."""
    alerts = []
    
    for event in events:
        for rule in rules:
            if event['eventName'] in rule['events']:
                # Check for policy patterns if specified
                if 'policy_patterns' in rule:
                    policy = str(event.get('requestParameters', {}))
                    if not any(p in policy for p in rule['policy_patterns']):
                        continue
                
                alerts.append({
                    "rule": rule['name'],
                    "event": event['eventName'],
                    "severity": rule['severity'],
                    "mitre": rule['mitre'],
                    "time": event['eventTime']
                })
    
    return alerts

# Detect threats
alerts = detect_cloud_threats(CLOUDTRAIL_EVENTS, DETECTION_RULES)
print("Detected Threats:")
for alert in alerts:
    print(f"  [{alert['severity']}] {alert['rule']}: {alert['event']} ({alert['mitre']})")

## 3. Multi-Cloud Normalization

In [ ]:
# Normalize events from different cloud providers

def normalize_aws_event(event: Dict) -> Dict:
    """Normalize AWS CloudTrail event."""
    return {
        "cloud": "AWS",
        "timestamp": event.get('eventTime'),
        "action": event.get('eventName'),
        "actor": event.get('userIdentity', {}).get('userName', 'unknown'),
        "source_ip": event.get('sourceIPAddress'),
        "resource": event.get('requestParameters', {}),
        "outcome": "success" if 'Success' in str(event.get('responseElements', '')) else "unknown"
    }

def normalize_azure_event(event: Dict) -> Dict:
    """Normalize Azure Activity Log event."""
    return {
        "cloud": "Azure",
        "timestamp": event.get('time'),
        "action": event.get('operationName'),
        "actor": event.get('caller'),
        "source_ip": event.get('callerIpAddress'),
        "resource": event.get('resourceId'),
        "outcome": event.get('status', {}).get('value', 'unknown')
    }

# Example normalized events
normalized = [normalize_aws_event(e) for e in CLOUDTRAIL_EVENTS]
print("Normalized Events:")
for n in normalized[:2]:
    print(json.dumps(n, indent=2))

## 4. LLM-Powered Cloud Security Analysis

In [ ]:
from anthropic import Anthropic

def analyze_cloud_activity(events: List[Dict]) -> str:
    """Use LLM to analyze cloud activity for threats."""
    
    client = Anthropic()
    
    prompt = f"""
    Analyze these AWS CloudTrail events for security threats.
    
    EVENTS:
    {json.dumps(events, indent=2)}
    
    Provide:
    1. Timeline of suspicious activity
    2. Likely attack scenario
    3. MITRE ATT&CK techniques used
    4. Recommended response actions
    5. Detection rules to prevent future attacks
    """
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=2000,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.content[0].text

# Uncomment to run with API key
# analysis = analyze_cloud_activity(CLOUDTRAIL_EVENTS)
# print(analysis)

## 5. Attack Pattern Detection

In [ ]:
# Common cloud attack patterns
ATTACK_PATTERNS = {
    "iam_persistence": {
        "sequence": ["CreateUser", "CreateAccessKey", "AttachUserPolicy"],
        "window_minutes": 30,
        "description": "Creation of backdoor IAM user with privileges"
    },
    "data_exfiltration": {
        "sequence": ["DescribeInstances", "ListBuckets", "GetObject"],
        "window_minutes": 60,
        "description": "Enumeration followed by data access"
    },
    "crypto_mining": {
        "sequence": ["RunInstances", "RunInstances", "RunInstances"],
        "window_minutes": 5,
        "description": "Rapid instance creation for mining"
    }
}

def detect_attack_pattern(events: List[Dict], patterns: Dict) -> List[Dict]:
    """Detect multi-step attack patterns."""
    detected = []
    event_names = [e['eventName'] for e in events]
    
    for pattern_name, pattern in patterns.items():
        sequence = pattern['sequence']
        
        # Check if sequence exists (simplified)
        matched = 0
        for seq_event in sequence:
            if seq_event in event_names:
                matched += 1
        
        if matched >= len(sequence) * 0.7:  # 70% match
            detected.append({
                "pattern": pattern_name,
                "description": pattern['description'],
                "confidence": matched / len(sequence)
            })
    
    return detected

patterns = detect_attack_pattern(CLOUDTRAIL_EVENTS, ATTACK_PATTERNS)
print("Detected Attack Patterns:")
for p in patterns:
    print(f"  {p['pattern']}: {p['description']} (confidence: {p['confidence']:.0%})")

## Key Takeaways

1. **CloudTrail Analysis**: Parse and analyze AWS audit logs
2. **Detection Rules**: Create rules for common cloud threats
3. **Multi-Cloud**: Normalize events across providers
4. **Attack Patterns**: Detect multi-step attacks

## Next Steps

- **Lab 20**: LLM Red Teaming - Attack AI systems with prompt injection, jailbreaking, and agentic exploits
- **CTF Challenges**: Test your skills on real-world scenarios
- **Capstone Projects**: Build production-ready security tools